In [1]:
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import re
import math

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

plt.style.use('seaborn-whitegrid')
matplotlib.rcParams['figure.figsize'] = (10.0, 4.0)

# Base Tank Class
This class is the tamplate that will hold the genetic code as its instruction set that can be run.


In [18]:
class Tank :  
    direction = ["north","east","south","west"] #for facing direction of tank
    dirRow = [0, 1, 0, -1] #used for moving and looking ahead of tank
    dirCol = [-1, 0, 1, 0] #used for moving and looking ahead of tank
    
    #energy cost for different actions
    forwCost = 10
    turnCost = 10
    fireCost = 30
    shieldCost = 20
    
    #height and width of the map the tank is simulated on
    gridHeight = 10
    gridWidth = 20
    
    def __init__(self, X, Y, facing) :
        self.X = X #starting x location
        self.Y = Y #starting y location
        self.facing = facing #starting facing direction (N: 0, E: 1, S: 2, W: 3)
        
        self.shields = False #if shields are powered on or not
        
        self.energy = 1000 #start with 1000 energy
        
    
    #base functions for the tank
    def Forw(self) : #move tank forward one step in the facing direction
        if (self.energy >= self.forwCost) : #only perform if the energy is available
            self.energy -= self.forwCost #subtract the cost from total energy
        
            self.X = (self.X + self.dirRow[self.facing]) % self.gridWidth
            self.Y = (self.Y + self.dirCol[self.facing]) % self.gridHeight

            print("Forw")
            print("   Coord: (" + str(self.X) + "," + str(self.Y) + ")")
    
    def TurnL(self) : #turn tank left
        if (self.energy >= self.turnCost) : #only perform if the energy is available
            self.energy -= self.turnCost #subtract the cost from total energy
        
            self.facing = (self.facing - 1) % 4 #rotate to previous cardinal directions and mod to reset if wrap

            print("TurnL")
            print("   Dir: " + self.direction[self.facing])
        
    def TurnR(self) : #turn tank right
        if (self.energy >= self.turnCost) : #only perform if the energy is available
            self.energy -= self.turnCost #subtract the cost from total energy

            self.facing = (self.facing + 1) % 4 #rotate to next cardinal directions and mod to reset if wrap

            print("TurnR")
            print("   Dir: " + self.direction[self.facing])
        
    def Fire(self) : #fire tank's cannon
        if (self.energy >= self.fireCost) : #only perform if the energy is available
            self.energy -= self.fireCost #subtract the cost from total energy
        
            #add logic for firing (range is 4 squares away in facing direcion)
        
        
        
    def ActivateShields(self) : #activate tank's shields, will have to rework benefits of shields with new goal
        if (self.energy >= self.shieldCost) : #only perform if the energy is available
            self.energy -= self.shieldCost #subtract the cost from total energy

            self.shields = True #turn on shields

            print("Shields")
            print("   On")
        
    def Wait(self) : #no-op
        pass #costs no energy
    
    #toString Function for printing
    def __str__(self) :
        return ("Coord: (" + str(self.X) + "," + str(self.Y) +
                ")\nDir: " + self.direction[self.facing] +
                "\nEnergy: " + str(self.energy))
        
        
#Test Tank class code
tank = Tank(2, 2, 3)
tank.Forw()
tank.Forw()
tank.Forw()
tank.Forw()
tank.TurnR()
tank.TurnR()
tank.Forw()
tank.Forw()
tank.Forw()
tank.Forw()


print(tank)

Forw
   Coord: (1,2)
Forw
   Coord: (0,2)
Forw
   Coord: (19,2)
Forw
   Coord: (18,2)
TurnR
   Dir: north
TurnR
   Dir: east
Coord: (18,2)
Dir: east
Energy: 0


# Deap GP Section
https://deap.readthedocs.io/en/master/tutorials/advanced/gp.html
Used for creating the genetic tank variants